In [1]:
import time
import numpy as np

import os
from datetime import date
from scipy.constants import elementary_charge, proton_mass 

import vis_canonical_fluxtubes as vis

import visit
visit.Launch()

True

In [3]:
database_prefix = '/Users/vonderlinden2/rsx_analysis/writing_to_vtk/output/2016-08-14-17-44'

In [4]:
today = str(date.today())
out_dir = '../output/' + today
try:
    os.makedirs(out_dir)
except:
    pass

output_path = out_dir + '/canonical_electron_fluxtube_'

vis.define_expressions(visit)
visit.OpenDatabase(database_prefix + '/Bdot_triple_probe_quantities*.vtk database')
field_nulls = np.loadtxt('../../centroid_fitting/output/2016-08-12/field_nulls.txt')
PseudocolorAtts, SliceAtts = vis.setup_current_pseudocolor(visit, max_val=5.1e5, min_val=0.0)
points_outer, points_inner = vis.launch_points(field_nulls[0])
AnnotationAtts = vis.setup_annotations(visit, time_scale=0.068)
#StreamlineAtts_line = vis.setup_field_line(visit, np.concatenate((field_nulls[0], [0.249])), outer_radius=0.01)

(electron_StreamlineAtts_outer, 
 electron_StreamlineAtts_inner) = vis.setup_massless_electron_canonical_flux_tubes(visit, points_outer, points_inner)

vis.set_default_view(visit)
visit.DrawPlots()

1L

In [5]:
visit.OpenGUI()

1L

In [6]:
save_atts = visit.SaveWindowAttributes()
save_atts.format = save_atts.PNG
save_atts.height = 1080
save_atts.width = 1920
save_atts.family = 0
visit.SetSaveWindowAttributes(save_atts)
ending='.png'

for time_point in xrange(0, 206):
    print time_point
    save_atts.fileName = output_path + str(time_point).zfill(4) + ending
    visit.SetSaveWindowAttributes(save_atts)
    
    points_outer, points_inner = vis.launch_points(field_nulls[time_point])
    #visit.SetActivePlots(1)
    #StreamlineAtts_line.pointSource = (field_nulls[time_point][0], field_nulls[time_point][1] + 0.01, 0.249)
    #visit.SetPlotOptions(StreamlineAtts_line)
    
    visit.SetActivePlots(1)
    electron_StreamlineAtts_outer.SetPointList(points_outer)
    visit.SetPlotOptions(electron_StreamlineAtts_outer)
    
    visit.SetActivePlots(2)
    electron_StreamlineAtts_inner.SetPointList(points_inner)
    visit.SetPlotOptions(electron_StreamlineAtts_inner)
    
    visit.SetTimeSliderState(time_point)
    
    name = visit.SaveWindow()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205


In [8]:
save_atts = visit.SaveWindowAttributes()
save_atts.format = save_atts.PNG
save_atts.height = 1080
save_atts.width = 1920
save_atts.family = 0
visit.SetSaveWindowAttributes(save_atts)
ending='.png'

for time_point in xrange(226, 250):
    print time_point
    save_atts.fileName = output_path + str(time_point - 20).zfill(4) + ending
    visit.SetSaveWindowAttributes(save_atts)
    
    points_outer, points_inner = vis.launch_points(field_nulls[time_point])
    
    #visit.SetActivePlots(1)
    #StreamlineAtts_line.pointSource = (field_nulls[time_point][0], field_nulls[time_point][1] + 0.01, 0.249)
    #visit.SetPlotOptions(StreamlineAtts_line)
    
    visit.SetActivePlots(1)
    electron_StreamlineAtts_outer.SetPointList(points_outer)
    visit.SetPlotOptions(electron_StreamlineAtts_outer)
    
    visit.SetActivePlots(2)
    electron_StreamlineAtts_inner.SetPointList(points_inner)
    visit.SetPlotOptions(electron_StreamlineAtts_inner)
    
    visit.SetTimeSliderState(time_point)
    
    name = visit.SaveWindow()

226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249


In [ ]:
save_atts = visit.SaveWindowAttributes()
save_atts.format = save_atts.PNG
save_atts.height = 1080
save_atts.width = 1920
save_atts.family = 0
visit.SetSaveWindowAttributes(save_atts)
ending='.png'

for time_point in xrange(visit.TimeSliderGetNStates()):
    print time_point
    save_atts.fileName = output_path + str(time_point).zfill(4) + ending
    visit.SetSaveWindowAttributes(save_atts)
    
    points_outer, points_inner = vis.launch_points(field_nulls[time_point])
    
    #visit.SetActivePlots(1)
    #StreamlineAtts_line.pointSource = (field_nulls[time_point][0], field_nulls[time_point][1] + 0.01, 0.249)
    #visit.SetPlotOptions(StreamlineAtts_line)
    
    visit.SetActivePlots(1)
    electron_StreamlineAtts_outer.SetPointList(points_outer)
    visit.SetPlotOptions(electron_StreamlineAtts_outer)
    
    visit.SetActivePlots(2)
    electron_StreamlineAtts_inner.SetPointList(points_inner)
    visit.SetPlotOptions(electron_StreamlineAtts_inner)
    
    visit.SetTimeSliderState(time_point)
    
    name = visit.SaveWindow()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161


In [17]:
!ffmpeg -framerate 30 -i ../output/2016-08-15/canonical_electron_fluxtube_%04d.png -r 30 -pix_fmt yuv420p \
 -vcodec libx264 -vf "scale=trunc(iw/2)*2:trunc(ih/2)*2" -y ../output/2016-08-15/out.mp4

ffmpeg version 2.8.1 Copyright (c) 2000-2015 the FFmpeg developers
  built with Apple LLVM version 6.0 (clang-600.0.56) (based on LLVM 3.5svn)
  configuration: --prefix=/Users/vonderlinden2/homebrew/Cellar/ffmpeg/2.8.1_1 --enable-shared --enable-pthreads --enable-gpl --enable-version3 --enable-hardcoded-tables --enable-avresample --cc=clang --host-cflags= --host-ldflags= --enable-opencl --enable-libx264 --enable-libmp3lame --enable-libvo-aacenc --enable-libxvid --enable-vda
  libavutil      54. 31.100 / 54. 31.100
  libavcodec     56. 60.100 / 56. 60.100
  libavformat    56. 40.101 / 56. 40.101
  libavdevice    56.  4.100 / 56.  4.100
  libavfilter     5. 40.101 /  5. 40.101
  libavresample   2.  1.  0 /  2.  1.  0
  libswscale      3.  1.101 /  3.  1.101
  libswresample   1.  2.101 /  1.  2.101
  libpostproc    53.  3.100 / 53.  3.100
Input #0, image2, from '../output/2016-08-15/canonical_electron_fluxtube_%04d.png':
  Duration: 00:00:07.67, start: 0.000000, bitrate: N/A
    Stream #0